In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [2]:
df = pd.read_excel('Training-testing.xlsx',sheet_name = 'Sheet2')
df.rename(columns = {'Date - Month':'Date'},inplace = True)
df.rename(columns = {'Current Status':'Current_Status'},inplace = True)
df = df.query("Current_Status not in ('Never Started','Leave of Absence')")
df.rename(columns = {'Current_Status':'Current Status'},inplace = True)
df.shape

(2999, 80)

In [410]:
df['Department'].value_counts()

Representative    2999
Name: Department, dtype: int64

In [370]:
values = df.query("Date in ('January' ,'February' , 'December')")
values['Position'].value_counts()

Representative    2209
Name: Position, dtype: int64

In [3]:
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [4]:
def preprocess_input(df):
    df = df.copy()
    
    # Filtring out null working hours
    df = df.query("Date in ('January' ,'February' , 'December')")
    df = df.fillna(0)
    # droping one value columns or null
    #df = df.drop(['Manager','Address 1','Last Working Day', 'Position', 'Position Group','Ident','Employee Name','Client','Market','Language','Location','Contract Type','Department','Termination Type','Termination Reason'], axis=1)
    df = df.drop(['University','Nationality','Address 1','Major'
                  ,'Last Working Day', 'Position', 'Position Group','Ident','Employee Name','Client',
                  'Market','Language','Location','Contract Type','Termination Type','Termination Reason',
                  'Sum of Internal Moves','Transfers %','QA Score','Coached Calls','Coached Required','Coaching Attainment',
                  'Avg. Time To Coach Hrs','Sum of Over-Time Duration','Utilization %','Productivity %',
                  'Canceled Actions'
                  ,'Approved Green','Canceled Green','Approved Yellow','Canceled Yellow','Approved Orange','Canceled Orange',
                  'Approved ACM Investigation','Canceled ACM Investigation','Approved Investigation','Canceled Investigation','Department'], axis=1)
   
    # Binary-encode binary columns
    df['Gender.'] = df['Gender.'].replace({'Female': 0, 'Male': 1})
    # Ordinal-encode
    df['Date'] = df['Date'].replace({'January': 1, 'February': 2,'December': 0,'March':3})
    df['Graduate / Undergrad'] = df['Graduate / Undergrad'].replace({'Drop Out': 0, 'Undergrad': 1,'Graduate': 2 , 'Gap Year':3})
    df['Current Status'] = df['Current Status'].replace({'Active': 1, 'Resigned': 0,'Terminated': 0,'Never Started':0,'Leave of Absence':0})
    # One-hot encoding
    for column in ['Program','Current Civil Status','District','Manager']:
        df = onehot_encode(df, column=column)
    
    
    
    # Split df into X and y
    y = df['Current Status']
    df = df.reindex(labels=Predict.columns,axis=1)
    df = df.fillna(0)
    X = df.drop('Current Status', axis=1)
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    # Scale X
    #scaler = StandardScaler()
    #scaler.fit(X_train)
    #X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    #X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    
    return X_train, X_test, y_train, y_test

In [5]:
def preprocess_input2(df):
    df = df.copy()
    # Filtring out null working hours
    df = df.query("Date == 'March'")
    df = df.fillna(0)
    df['Gender.'] = df['Gender.'].replace({'Female': 0, 'Male': 1})
    # Ordinal-encode
    #df['Graduate / Undergrad'] = df['Graduate / Undergrad'].replace({'Drop Out': 0, 'Undergrad': 1,'Graduate': 2 , 'Gap Year':3})
    #df['Current Status'] = df['Current Status'].replace({'Active': 1, 'Resigned': 0,'Terminated': 0})
    df = df.drop(['University','Nationality','Major','Address 1','Last Working Day', 'Position', 'Position Group','Ident','Employee Name','Client',
                  'Market','Language','Location','Contract Type','Termination Type','Termination Reason',
                  'Sum of Internal Moves','Transfers %','QA Score','Coached Calls','Coached Required','Coaching Attainment',
                  'Avg. Time To Coach Hrs','Sum of Over-Time Duration','Utilization %','Productivity %','Canceled Actions'
                  ,'Approved Green','Canceled Green','Approved Yellow','Canceled Yellow','Approved Orange','Canceled Orange',
                  'Approved ACM Investigation','Canceled ACM Investigation','Approved Investigation','Canceled Investigation','Department'], axis=1)
 
    # Ordinal-encode
    df['Date'] = df['Date'].replace({'January': 1, 'February': 2,'December': 0,'March':3})
    df['Graduate / Undergrad'] = df['Graduate / Undergrad'].replace({'Drop Out': 0, 'Undergrad': 1,'Graduate': 2 , 'Gap Year':3})
    # One-hot encoding
    for column in ['Program','Current Civil Status','District','Manager']:
        df = onehot_encode(df, column=column)
    return df

In [6]:
Predict = preprocess_input2(df)
X_train, X_test, y_train, y_test = preprocess_input(df)
X_train

,Date - Year,Date,Sum of Salary,Gender.,Age,Graduate / Undergrad,AHT,Sum of Handled Transactions,Sum of Total Surveys,NPS %,CSAT %,DSAT %,FCR %,QA Monitored,"QA Critical Pass % (EUC, BC, CC, ZTP)",QA EUC Pass %,QA BC Pass %,QA CC Pass %,QA ZTP Pass %,Other Monitored Transactions,Positive Coaching,Sum of Working Hours On Premises,Sum of Working Hours WAHA,Sum of Over-Time Days,Sum of Working During Holdidays,Occupancy %,ABS %,Sum of Paid Leaves,Sum of Unpaid Leaves,Sum of Sick-Leaves,Sum of No Shows,Sum of Attrition,Approved Actions,Approved Red,Canceled Red,Approved Black,Canceled Black,Program_Expedia PLAZA,Program_Expedia PLAZA ECA Chat,Program_Expedia PLAZA ECA Chat EN,Program_Expedia PLAZA ECA Chat FR,Program_Expedia PLAZA ECA EN Sales,Program_Expedia PLAZA ECA EN T2,Program_Expedia PLAZA ECA Elite,Program_Expedia PLAZA ECA En Service,Program_Expedia PLAZA ECA FR Sales,Program_Expedia PLAZA ECA FR Service,Program_Expedia PLAZA ECA FR T2,Program_Expedia PLAZA ECA PPV,Program_Expedia PLAZA Lodging Agent,Program_Expedia PLAZA New Hire CA,Program_Expedia PLAZA New Hire OPM,Program_Expedia PLAZA Non-Lodging Agent,Program_Expedia PLAZA OPM EN Sales,Program_Expedia PLAZA OPM EN Service,Program_Expedia PLAZA OPM French,Program_Expedia PLAZA Universal Chat,Program_Expedia PLAZA Universal Voice,Current Civil Status_Married,Current Civil Status_Single,Current Civil Status_Unknown,District_ El Sahel,District_ Faisl,District_ Giza,District_ Kafr el Sheikh,District_ Old cairo,District_ Sahel,District_-,District_10th of Ramadan City 1,District_15 May City,"District_15th,Of-May-",District_1st Settlement,District_6th of October,District_6th of October City 1,District_6th of October City 2,District_Abdeen,District_Agouza,District_Ain Shames,District_Ain Shams,District_Al Montazah,District_Al Montazah 1,District_Al Montazah 2,District_Al Raml,District_Al Sayeda Zainab,District_Al-Dakahlya,District_Alexandria,District_Alf Maskan,District_Amreya,District_Aserat,District_Ashmoun,District_Aswan,District_Atfeh,District_Azbakia,District_Bab El Shariaa,District_Bab Sharqi,District_Banha,District_Bulaq,District_Bulaq el Dakrur,District_Daher,District_Damietta,District_Dar El Salam,District_Dokki,District_El Agouza,District_El Arab,District_El Ayyat,District_El Badrashein,District_El Basatin,District_El Darb El Ahmar,District_El Dokki,District_El Gamaliya,District_El Haram,District_El Hawamdeya,District_El Maadi,District_El Mahalla El Kubra 1,District_El Mahalla El Kubra 2,District_El Manial,District_El Mansoura,District_El Mansoura 1,District_El Mansoura1,District_El Marg,District_El Matareya,District_El Mataria,District_El Nozha,District_El Omraniya,District_El Qobba,District_El Qobbah,District_El Raml 2,District_El Sahel,District_El Sayeda Zeinab,District_El Sharabiya,District_El Sharkia,District_El Sharq,District_El Shikh Zayed,District_El Shorouk,District_El Shoruk,District_El Ubour,District_El Weili,District_El Weily,District_El Zaher,District_El Zawya El Hamra,District_El Zaytoun,District_El-Mansheya,District_El-Montzah,District_El-Nozha,District_El-Salam,District_El-Zaytoun,District_El-darb-El-ahmar,District_ElFayoum,District_ElMaddi,District_Elmarg,District_Elmenia,District_Elsahel,District_Elwarak,District_Embaba,District_Faiyum 1,District_Faysl,District_Fifth Settlement,District_Fifth Settlment,District_Garden City,District_Gharbia,District_Hadayek El Koba,District_Hadayek El kobba,District_Helioplis,District_Heliopolis,District_Helwan,District_Imababa,District_Imbaba,District_Ismailia,District_Ismailia 3,District_Maadi,District_Manshiyat Naser,District_Mansoura,District_Masr Al Jadida,District_Masr El Gededa,District_Masr Elkadema,District_Mataria,District_Matrya,District_Mokattam,District_Nasr City,District_Nasr City 1,District_Nasr City 2,District_New Cairo 1,District_New Cairo 2,District_New Cairo 3,District_Nozha,District_Nozha El Gdeeda,District_Obour,District_October,District_Omarnia,District_Omrania,District_Ossim,District_Other,Di

In [7]:
models = {
    "                   Logistic Regression": LogisticRegression(),
    "                   K-Nearest Neighbors": KNeighborsClassifier(),
    "                         Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (Linear Kernel)": LinearSVC(),
    "   Support Vector Machine (RBF Kernel)": SVC(),
    "                        Neural Network": MLPClassifier(),
    "                         Random Forest": RandomForestClassifier(),
    "                     Gradient Boosting": GradientBoostingClassifier()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")
    
for name, model in models.items():
    print(name + ": {:.2f}%".format(model.score(X_test, y_test) * 100))
    
Predict = Predict.drop(columns=['Current Status'], axis = 1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


                   Logistic Regression trained.
                   K-Nearest Neighbors trained.
                         Decision Tree trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                        Neural Network trained.
                         Random Forest trained.
                     Gradient Boosting trained.
                   Logistic Regression: 81.15%
                   K-Nearest Neighbors: 81.30%
                         Decision Tree: 81.30%
Support Vector Machine (Linear Kernel): 81.60%
   Support Vector Machine (RBF Kernel): 81.15%
                        Neural Network: 81.45%
                         Random Forest: 86.43%
                     Gradient Boosting: 86.27%


In [8]:
Predict = Predict.drop(columns=['Test'], axis = 1)

KeyError: "['Test'] not found in axis"

In [9]:
log_mod = LogisticRegression()
log_mod.fit(X_train, y_train)
Predict['Test'] = log_mod.predict(Predict)
df['Outcome'] = Predict['Test']
Predict['Test'].mean(),Predict['Test'].value_counts()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.950632911392405,
 1    751
 0     39
 Name: Test, dtype: int64)

In [363]:
Predict = Predict.drop(columns=['Test'], axis = 1)
log_mod = KNeighborsClassifier()
log_mod.fit(X_train, y_train)
Predict['Test'] = log_mod.predict(Predict)
df['Outcome'] = Predict['Test']
Predict['Test'].mean(),Predict['Test'].value_counts()

(0.9455696202531646,
 1    747
 0     43
 Name: Test, dtype: int64)

In [350]:
Predict = Predict.drop(columns=['Test'], axis = 1)
Neural = DecisionTreeClassifier()
Neural.fit(X_train, y_train)
Predict['Test'] = Neural.predict(Predict)
df['Outcome'] = Predict['Test']
Predict['Test'].mean(),Predict['Test'].value_counts()

(0.8645569620253165,
 1    683
 0    107
 Name: Test, dtype: int64)

In [351]:
Predict = Predict.drop(columns=['Test'], axis = 1)
Neural = SVC()
Neural.fit(X_train, y_train)
Predict['Test'] = Neural.predict(Predict)
df['Outcome'] = Predict['Test']
Predict['Test'].mean(),Predict['Test'].value_counts()

(1.0,
 1    790
 Name: Test, dtype: int64)

In [352]:
Predict = Predict.drop(columns=['Test'], axis = 1)
Neural = MLPClassifier()
Neural.fit(X_train, y_train)
Predict['Test'] = Neural.predict(Predict)
df['Outcome'] = Predict['Test']
Predict['Test'].mean(),Predict['Test'].value_counts()

(0.9848101265822785,
 1    778
 0     12
 Name: Test, dtype: int64)

In [353]:
Predict = Predict.drop(columns=['Test'], axis = 1)
Neural = RandomForestClassifier()
Neural.fit(X_train, y_train)
Predict['Test'] = Neural.predict(Predict)
df['Outcome'] = Predict['Test']
Predict['Test'].mean(),Predict['Test'].value_counts()

(0.9620253164556962,
 1    760
 0     30
 Name: Test, dtype: int64)

In [354]:
Predict = Predict.drop(columns=['Test'], axis = 1)
Neural = GradientBoostingClassifier()
Neural.fit(X_train, y_train)
Predict['Test'] = Neural.predict(Predict)
df['Outcome'] = Predict['Test']
Predict['Test'].mean(),Predict['Test'].value_counts()

(0.9582278481012658,
 1    757
 0     33
 Name: Test, dtype: int64)

In [ ]:
df['']

In [10]:
df.to_excel('Expedia Risk List.xlsx',index = False)